In [ ]:
# Dependancies. Uncomment to automatically install.
# %pip install rosbags
# %pip install pandas
# %pip install numpy
# %pip install matplotlib

In [ ]:
import utils.rosbags_converter as rc
from utils.seatrac_enums import CST_E
import utils.plotter_utils as p_utils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

# Settings
- RELOAD: If true, converts rosbags. Otherwise assumes rosbags are already converted
- ROSBAGS_DIR: Directory where your rosbags are stored. Can contain multiple bags
- SAVES_DIR: The directory to save converted csv files (to save time reloading)
- VERBOSE: Print non-essential values

In [ ]:
RELOAD = False
VERBOSE = False
ROSBAGS_DIR = "../../bag"
SAVES_DIR = ROSBAGS_DIR+"/converted_bags/coug2"
ROSMSGS_DIR = "../../cougars-ros2/src"

In [ ]:
# Convert Rosbags

if RELOAD:
    print("converting rosbags")
    dataframes = p_utils.get_dataframes(
        rosbags_dir=ROSBAGS_DIR, rosmsgs_dir=ROSMSGS_DIR, csv_dir=SAVES_DIR,
        keywords=None, topics=None,verbose=VERBOSE)
else:
    print("loading dataframes")
    dataframes = rc.load_dataframes(SAVES_DIR, keywords=None, verbose=VERBOSE)
    if len(dataframes)==0:
        raise RuntimeError("Lenth of dataframes is 0. Dataframes may not be loaded")
print("dataframes loaded")

---

In [ ]:
# converts ros header.stamp.sec and header.stamp.nanosec into pandas Timestamps objects
# and adds a new column "timestamp" to 
p_utils.insert_timestamps(dataframes)

In [ ]:
# GPS
for path, bag in dataframes.items():
    print(path)

    # Print GPS timeline. Green = Good fix. Orange = bad fix.
    gps_fix = p_utils.get_topic(bag, "/extended_fix")
    if gps_fix is None: continue
    good_gps_mask = gps_fix["status.status"]!=-1
    good_gps_ts = gps_fix["timestamp"][good_gps_mask]
    ok_gps = gps_fix["timestamp"][gps_fix["latitude"]!=0]
    fig, ax = plt.subplots(figsize=(8, .3))
    ax.plot(ok_gps, [0]*len(ok_gps), 'o', color="orange", markersize='3')
    ax.plot(good_gps_ts, [0]*len(good_gps_ts), 'go', markersize='3')
    plt.show()
    
    gps_odom = p_utils.get_topic(bag, "/gps_odom")
    good_gps_odom = gps_odom[gps_odom["timestamp"].isin(good_gps_ts)]
    if gps_odom is None: continue
    fig, ax = plt.subplots()
    p_utils.plot_pose_w_cov(gps_odom, ax=ax)
    plt.show()

In [ ]:
import utils.plotter_utils as p_utils
# DVL path
# for path, bag in dataframes.items():
#     print(path)
#     for topic in bag.keys():
#         if topic.endswith("/dvl/position"):
#             dvl_df = bag[topic]
#             plt.plot(dvl_df["position.x"], dvl_df["position.y"])
#             plt.plot([dvl_df["position.x"].iloc[0]], [dvl_df["position.y"].iloc[0]], 'o')
#             plt.show()

for path, bag in dataframes.items():
    print(path)
    dvl_odom = p_utils.get_topic(bag, "/dvl/dead_reckoning")
    if dvl_odom is None: continue
    ax = p_utils.plot_pose_w_cov(dvl_odom)

In [ ]:
ax = p_utils.plot_mahalanobis_ellipse(1,2,[[1,0],[0,1]])
p_utils.plot_mahalanobis_ellipse(1,-2,[[1,0],[0,1]], ax=ax)
plt.show()